In [1]:
import seaborn as sns
import numpy as np
import logging
from itertools import product
from copy import deepcopy, copy
from collections.abc import Sequence

In [2]:
from cosmicfishpie.fishermatrix import cosmicfish
from cosmicfishpie.LSSsurvey import photo_obs as pobs
from cosmicfishpie.LSSsurvey import photo_cov as pcov
from cosmicfishpie.likelihood import PhotometricLikelihood
from cosmicfishpie.utilities.utils import printing as upr
from nautilus import Prior
from nautilus import Sampler
import re
import time

In [3]:
snscolors = sns.color_palette("colorblind")
def is_indexable_iterable(var):
    return isinstance(var, (list, np.ndarray, Sequence)) and not isinstance(var, (str, bytes))


In [4]:
logger = logging.getLogger("cosmicfishpie.cosmology.nuisance")
logger.setLevel(logging.INFO)

In [5]:
upr.debug = False

In [6]:
upr.debug_print("test")

In [7]:
outroot = "cosmicjellyfish_Euclid-ISTF-Pess-GCphoto_symb_simple"

In [8]:
sampler_settings = {
    "n_live": 1000,
    "n_networks": 8,
    "n_batch": 256,
    "pool": 8,
}

In [9]:
fiducial = {
    "Omegam": 0.3145714273,
    "Omegab": 0.0491989,
    "h": 0.6737,
    "ns": 0.96605,
    "sigma8": 0.81,
    "w0": -1.0,
    "wa": 0.0,
    "mnu": 0.06,
    "Neff": 3.044,
}
observables = ['GCph']

In [10]:
options = {
    "accuracy": 1,
    "feedback": 1,
    "code": "symbolic",
    "outroot": outroot,
    "survey_name": "Euclid",
    "survey_name_photo": "Euclid-Photometric-ISTF-Pessimistic",
    "survey_name_spectro": False,
    "specs_dir": "../cosmicfishpie/configs/default_survey_specifications/",
    "cosmo_model": "LCDM",
}
cosmoFM_fid = cosmicfish.FisherMatrix(
    fiducialpars=fiducial,
    options=options,
    observables=observables,
    cosmoModel=options["cosmo_model"],
    surveyName=options["survey_name"],
)

****************************************************************
   _____               _     _____     __  
  / ___/__  ___ __ _  (_)___/ __(_)__ / /  
 / /__/ _ \(_-</  ' \/ / __/ _// (_-</ _ \ 
 \___/\___/___/_/_/_/_/\__/_/ /_/___/_//_/ 

****************************************************************
 This is the new Python version of the CosmicFish code.
****************************************************************

  -> Survey loaded:  Euclid-Photometric-ISTF-Pessimistic

  -> Computing cosmology at the fiducial point

  ---> Cosmological functions obtained in:   0.11 s


In [11]:
cosmoFM_fid.freeparams

{'Omegam': 0.01,
 'Omegab': 0.01,
 'h': 0.01,
 'ns': 0.01,
 'sigma8': 0.01,
 'b1': 0.06,
 'b2': 0.06,
 'b3': 0.06,
 'b4': 0.06,
 'b5': 0.06,
 'b6': 0.06,
 'b7': 0.06,
 'b8': 0.06,
 'b9': 0.06,
 'b10': 0.06}

In [12]:
# #photo_fid = pobs.ComputeCls(cosmopars=cosmoFM_fid.fiducialcosmopars,
#                             photopars=cosmoFM_fid.photopars,
#                             IApars=cosmoFM_fid.IApars,
#                             biaspars=cosmoFM_fid.photobiaspars)

# #photo_fid.compute_all()

# #photo_cov_fid = pcov.PhotoCov(cosmopars=cosmoFM_fid.fiducialcosmopars,
#     #photopars=cosmoFM_fid.photopars,
#     #IApars=cosmoFM_fid.IApars,
#     #biaspars=cosmoFM_fid.photobiaspars,
#     #fiducial_Cls=photo_fid)

In [13]:
cosmoFM_fid.allparams

{'Omegam': 0.3145714273,
 'Omegab': 0.0491989,
 'h': 0.6737,
 'ns': 0.96605,
 'sigma8': 0.81,
 'w0': -1.0,
 'wa': 0.0,
 'mnu': 0.06,
 'Neff': 3.044,
 'bias_model': 'binned',
 'b1': np.float64(1.0997727037892875),
 'b2': np.float64(1.220245876862528),
 'b3': np.float64(1.2723993083933989),
 'b4': np.float64(1.316624471897739),
 'b5': np.float64(1.35812370570578),
 'b6': np.float64(1.3998214171814918),
 'b7': np.float64(1.4446452851824907),
 'b8': np.float64(1.4964959071110084),
 'b9': np.float64(1.5652475842498528),
 'b10': np.float64(1.7429859437184225),
 'fout': 0.1,
 'co': 1,
 'cb': 1,
 'sigma_o': 0.05,
 'sigma_b': 0.05,
 'zo': 0.1,
 'zb': 0.0,
 'IA_model': 'eNLA',
 'AIA': 1.72,
 'betaIA': 2.17,
 'etaIA': -0.41}

In [14]:
cosmoFM_fid.observables

['GCph']

In [ ]:
#fishmat_photo = cosmoFM_fid.compute()


In class: FisherMatrix  ----> Computing photo Fisher matrix

Computing fiducial

Fiducial generated in 0.55 s

Noise added to fiducial

Noisy Cls generated in 0.00 s

Computed covariance matrix

Covmat of Cls generated in 0.13 s

Total calculation in 0.69 s
>> computing derivs >>

  +++ Computing derivative on Omegam

In class: derivatives  Derivative on Omegam done! in :  1.31 s

  +++ Computing derivative on Omegab

In class: derivatives  Derivative on Omegab done! in :  1.30 s

  +++ Computing derivative on h

In class: derivatives  Derivative on h done! in :  1.24 s

  +++ Computing derivative on ns

In class: derivatives  Derivative on ns done! in :  1.21 s

  +++ Computing derivative on sigma8

In class: derivatives  Derivative on sigma8 done! in :  1.22 s

  +++ Computing derivative on b1

In class: derivatives  Derivative on b1 done! in :  1.04 s

  +++ Computing derivative on b2

In class: derivatives  Derivative on b2 done! in :  1.02 s

  +++ Computing derivative on b3

In 

In [ ]:
photo_like = PhotometricLikelihood(cosmo_data=cosmoFM_fid, cosmo_theory=cosmoFM_fid)

In [17]:
#print(Cells_fid["Cell_LL"].shape)
#print(Cells_fid["Cell_GG"].shape)
#print(Cells_fid["Cell_GL"].shape)

In [18]:
ellmax_GC = cosmoFM_fid.specs["lmax_GCph"]
print(ellmax_GC)

750


In [19]:
samp1dic = cosmoFM_fid.allparams.copy()

In [20]:
print("Sample likelihood", photo_like.loglike(param_dict=samp1dic))
samp1dic = cosmoFM_fid.allparams

samp1dic['b1'] = 1.0

print("Sample likelihood", photo_like.loglike(param_dict=samp1dic))

samp1dic['h'] = 0.7

print("Sample likelihood", photo_like.loglike(param_dict=samp1dic))

Sample likelihood 4.330972755260946e-11
Sample likelihood -1568.703678015382
Sample likelihood -1522.4374216102938


In [21]:
prior_nonuis = Prior()
prior_withnuis = Prior()

In [22]:
prior_dict ={
 'Omegam': [0.24, 0.4],
 'Omegab': [0.04, 0.06],
 'h': [0.61, 0.75],
 'ns': [0.92, 1.00],
 'sigma8': [0.79, 0.83],
 'AIA': [1.0, 3.0],
 'etaIA' :[-6.0, 6.0],
 'b1': [1.0, 3.0],
 'b2': [1.0, 3.0],
 'b3': [1.0, 3.0],
 'b4': [1.0, 3.0],
 'b5': [1.0, 3.0],
 'b6': [1.0, 3.0],
 'b7': [1.0, 3.0],
 'b8': [1.0, 3.0],
 'b9': [1.0, 3.0],
 'b10': [1.0, 3.0]
 }

In [23]:
cosmoFM_fid.freeparams

{'Omegam': 0.01,
 'Omegab': 0.01,
 'h': 0.01,
 'ns': 0.01,
 'sigma8': 0.01,
 'b1': 0.06,
 'b2': 0.06,
 'b3': 0.06,
 'b4': 0.06,
 'b5': 0.06,
 'b6': 0.06,
 'b7': 0.06,
 'b8': 0.06,
 'b9': 0.06,
 'b10': 0.06}

In [24]:
for par in prior_dict.keys():
    if par in cosmoFM_fid.freeparams.keys():
        dist_prior = (prior_dict[par][0], prior_dict[par][1])
        if re.match(r'b\d+', par):
            prior_withnuis.add_parameter(par, dist_prior)
        elif re.search(r'IA', par):
            prior_withnuis.add_parameter(par, dist_prior)
        else:
            prior_nonuis.add_parameter(par, dist_prior)
            prior_withnuis.add_parameter(par, dist_prior)

In [25]:
print(prior_nonuis.keys)
print(prior_withnuis.keys)

['Omegam', 'Omegab', 'h', 'ns', 'sigma8']
['Omegam', 'Omegab', 'h', 'ns', 'sigma8', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10']


In [26]:
use_nuisance = True

In [27]:
if use_nuisance: 
    prior_chosen = prior_withnuis
else:
    prior_chosen = prior_nonuis
print("Loading prior with keys: ", prior_chosen.keys)

Loading prior with keys:  ['Omegam', 'Omegab', 'h', 'ns', 'sigma8', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10']


In [28]:
tini = time.time()
print("Starting sampler at", time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tini)))

Starting sampler at 2025-09-29 13:00:47


In [29]:
sampler = Sampler(prior_chosen, 
                          photo_like.loglike, 
                          n_live=sampler_settings["n_live"], 
                          n_networks=sampler_settings["n_networks"], 
                          n_batch=sampler_settings["n_batch"], 
                          pool=sampler_settings["pool"], 
                          pass_dict=False,
                          filepath=options["outroot"]+".hdf5", 
                          resume=True,
                          likelihood_kwargs={'prior': prior_chosen}
                          )
sampler.run(verbose=True, discard_exploration=True)
log_z_all = sampler.evidence()
print('Evidence:', log_z_all)
points_all, log_w_all, log_l_all = sampler.posterior()

Starting the nautilus sampler...
Please report issues at github.com/johannesulf/nautilus.
Status    | Bounds | Ellipses | Networks | Calls    | f_live | N_eff | log Z    


Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/casas/miniforge3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/casas/miniforge3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/casas/miniforge3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/casas/miniforge3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/casas/miniforge3/lib/python3.12/multiprocessing/proce

KeyboardInterrupt: 

In [ ]:
tfin = time.time()
elapsed = tfin - tini
hours = int(elapsed // 3600)
minutes = int((elapsed % 3600) // 60)
seconds = int(elapsed % 60)
print("Sampler finished at", time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tfin)))
print(f"Total time elapsed: {hours:02d}:{minutes:02d}:{seconds:02d}")

In [ ]:
sample_wghlkl = (np.vstack((points_all.T, np.exp(log_w_all), log_l_all)).T)

In [ ]:
outfile_chain = options["outroot"]+".txt"
print(f"Saving chain to text file {outfile_chain}")

In [ ]:
headerlist = ['loglike', 'weights'] + list(prior_chosen.keys)
header = " ".join(headerlist)
print("Saving header: ", header)

In [ ]:
np.savetxt(outfile_chain, sample_wghlkl, header=header)